In [1]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import csv

In [2]:
def read_data_ict(campaign, tscol="Time_Start", cols_to_keep=[], extra_variables = [], delim_whitespace=False, file_pattern = "*.ict"):

    path = campaign + "/"
    
    df_list = []
    
    for i,file in enumerate(glob.glob(path+file_pattern)):

        lineStart = pd.read_csv(file, nrows = 1, header=None, delim_whitespace=delim_whitespace, encoding='latin1')[0][0] - 1

        single_flight_data = pd.read_csv(file, skiprows = lineStart,delim_whitespace=delim_whitespace, encoding='latin1')
            
        # get fillValue
        fillValue = pd.read_csv(file, skiprows = 11, nrows = 1, header = None, delim_whitespace=delim_whitespace, encoding='latin1')[0][0]

        # replace fillValue with NA
        single_flight_data[single_flight_data == fillValue] = np.nan
        
        # read in date of flight
        read_date = pd.read_csv(file, skiprows = 6, nrows = 1, header = None, delim_whitespace=delim_whitespace, encoding='latin1')
        date_info = [read_date[0][0],read_date[1][0],read_date[2][0]]
        flight_date = '-'.join([str(n) for n in date_info])
        
        # update timestamp to datetime -> timestamp is seconds from beginning of day
        time_col_name = tscol
        single_flight_data["timestamp"] = pd.to_datetime(single_flight_data[time_col_name], unit='s', origin=pd.Timestamp(flight_date))#datetime.utcfromtimestamp(single_flight_data[' UTC']).strftime('%Y-%m-%d %H:%M:%S')
        
        df_list.append(single_flight_data)
        
    # concat all the single flight dataframes into one
    campaign_data = pd.concat(df_list)
    campaign_data.set_index("timestamp", drop=True, inplace=True)
    campaign_data.sort_index(inplace=True)
    
    # remove special character
    campaign_data.columns = campaign_data.columns.str.replace(' ', '')
     
    return campaign_data

In [ ]:
glob.glob(path+file_pattern)

In [4]:
file = '/Users/joepalmo/Downloads/firexaq-mrg60-dc8_merge_20190722_R1.ict'

In [5]:
delim_whitespace= False

tscols = ["UTC", "UTC_mid", "Time_Start", "UTC_Start"]

lineStart = pd.read_csv(file, nrows = 1, header=None, delim_whitespace=delim_whitespace, encoding='latin1')[0][0] - 1

single_flight_data = pd.read_csv(file, skiprows = lineStart,delim_whitespace=delim_whitespace, encoding='latin1')

tscol = list(set(tscols).intersection(single_flight_data.columns.to_list()))[0]

In [19]:
list(set(tscols).intersection(single_flight_data.columns.to_list()))[0]

'Time_Start'

In [17]:
{'Time_Start'}

{'Time_Start'}

In [42]:
def read_data_ict(campaign, tscols=["UTC", "UTC_mid", "Time_Start", "UTC_Start"], delim_whitespace=False, ):
    lineStart = pd.read_csv(file, nrows = 1, header=None, delim_whitespace=delim_whitespace, encoding='latin1')[0][0] - 1

    data = pd.read_csv(file, skiprows = lineStart,delim_whitespace=delim_whitespace, encoding='latin1')
        
    # get fillValue
    fillValue = pd.read_csv(file, skiprows = 11, nrows = 1, header = None, delim_whitespace=delim_whitespace, encoding='latin1')[0][0]

    # replace fillValue with NA
    data[data == fillValue] = np.nan
    
    # read in date of flight
    read_date = pd.read_csv(file, skiprows = 6, nrows = 1, header = None, delim_whitespace=delim_whitespace, encoding='latin1')
    date_info = [read_date[0][0],read_date[1][0],read_date[2][0]]
    flight_date = '-'.join([str(n) for n in date_info])
    
    # update timestamp to datetime -> timestamp is seconds from beginning of day
    time_col_name = list(set(tscols).intersection(data.columns.to_list()))[0]
    data["timestamp"] = pd.to_datetime(data[time_col_name], unit='s', origin=pd.Timestamp(flight_date))
    
    data.set_index("timestamp", drop=True, inplace=True)
    data.sort_index(inplace=True)

    #get lat long columns
    for x in data.columns.to_list():
        if 'latitude' in x.lower():
            data.rename(columns={x:"latitude"}, inplace=True)
            break
        elif ('_lat' in x.lower()):
            data.rename(columns={x:"latitude"}, inplace=True)
            break

    for x in data.columns.to_list():
        if 'longitude' in x.lower():
            data.rename(columns={x:"longitude"}, inplace=True)
            break
        elif ('_lon' in x.lower()):
            data.rename(columns={x:"longitude"}, inplace=True)
            break

    return data

In [43]:
test_data = read_data_ict(file)

/var/folders/h_/8fdbb_yn5695gpckxmlvgrj00000gn/T/ipykernel_74617/3321459940.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["timestamp"] = pd.to_datetime(data[time_col_name], unit='s', origin=pd.Timestamp(flight_date))


In [44]:
test_data

,Time_Start,Time_Stop,Day_Of_Year_YANG,latitude,longitude,MSL_GPS_Altitude_YANG,HAE_GPS_Altitude_YANG,Pressure_Altitude_YANG,Radar_Altitude_YANG,Ground_Speed_YANG,...,jBrONO2_BrO_NO2_CAFS_HALL,jBrONO2_Br_NO3_CAFS_HALL,jBrCl_Br_Cl_CAFS_HALL,jCHBr3_NoProductsSpecified_CAFS_HALL,smoke_age_HOLMES,smoke_age_corr_HOLMES,smoke_age_rise_HOLMES,smoke_rise_HOLMES,fire_distance_HOLMES,smoke_agemethod_HOLMES
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-07-22 18:17:30,65850,65910,203,34.607087,-118.152976,1272.4700,1242.74,3960.50,1497.2800,109.0077,...,0.000179,0.001015,0.009425,1.165000e-06,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-22 18:18:30,65910,65970,203,34.658115,-118.170044,1553.9500,1524.22,4820.37,2618.4900,117.9437,...,0.000162,0.000916,0.008420,1.130000e-06,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-22 18:19:30,65970,66030,203,34.683717,-118.110757,1642.1200,1613.44,5083.18,2979.0800,121.0243,...,0.000145,0.000824,0.007515,1.009000e-06,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-22 18:20:30,66030,66090,203,34.661254,-118.034263,2016.3800,1986.74,6207.30,4144.4900,130.0288,...,0.000235,0.001333,0.012356,1.581000e-06,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-22 18:21:30,66090,66150,203,34.632011,-117.955608,2076.6800,2046.91,6396.50,4233.6400,132.5478,...,0.000195,0.001106,0.009997,1.370000e-06,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-23 01:03:30,90210,90270,203,43.411588,-116.195437,2004.0500,1984.78,6257.68,3579.3600,137.0425,...,0.000141,0.000798,0.008570,4.084000e-07,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-23 01:04:30,90270,90330,203,43.459803,-116.134011,1774.3700,1753.01,5571.00,2567.3100,113.2422,...,0.000133,0.000757,0.008169,3.621000e-07,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-23 01:05:30,90330,90390,203,43.511321,-116.117520,1395.2300,1374.91,4423.12,1346.3900,92.7857,...,0.000122,0.000691,0.007525,3.161000e-07,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
list(set(['longitude']).intersection(test_data.columns.to_list().lower()))[0]

AttributeError: 'list' object has no attribute 'lower'

In [36]:
for x in test_data.columns.to_list():
    if 'latitude' in x.lower():
        print(x)
    elif ('_lat' in x.lower()):
        print(x)

 Latitude_YANG


In [38]:
for x in test_data.columns.to_list():
    if 'longitude' in x.lower():
        print(x)
    elif ('_lon' in x.lower()):
        print(x)

 Longitude_YANG
